# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

## Откройте и изучите файл

In [2]:
df = pd.read_csv('Ya_data/users_behavior.csv')
df.sample(10, random_state = 12345)

,calls,minutes,messages,mb_used,is_ultra
1415,82.0,507.89,88.0,17543.37,1
916,50.0,375.91,35.0,12388.40,0
1670,83.0,540.49,41.0,9127.74,0
686,79.0,562.99,19.0,25508.19,1
2951,78.0,531.29,20.0,9217.25,0
654,53.0,478.18,78.0,20152.53,0
2827,73.0,582.47,33.0,12095.91,0
1466,31.0,172.10,25.0,31077.59,0
2223,28.0,222.21,30.0,22986.30,0
2639,68.0,523.56,14.0,18910.66,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В датасете 3213 объектов. Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.  
Известно:  
сalls — количество звонков,  
minutes — суммарная длительность звонков в минутах,  
messages — количество sms-сообщений,  
mb_used — израсходованный интернет-трафик в Мб,  
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  
Пропусков нет.

## Разбейте данные на выборки

In [4]:
#выделим целевой признак
features = df.drop('is_ultra', axis = 1)
target = df['is_ultra']
# разобьем нанные на 3 части: обучающая, валидационную и тестовую выборки в соотношнии 3:1:1

features_train, features_test_valid, target_train, target_test_valid = train_test_split(features, target, 
                                                                            test_size=0.4, 
                                                                            random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_test_valid, target_test_valid, 
                                                                            test_size=0.5, 
                                                                            random_state=12345)


print('Обучающие объекты: признаки {}, цель {}'.format(features_train.shape, target_train.shape))
print('Валидационные объекты: признаки {}, цель {}'.format(features_valid.shape, target_valid.shape))
print('Тестовые объекты: признаки {}, цель {}'.format(features_test.shape, target_test.shape))

Обучающие объекты: признаки (1928, 4), цель (1928,)
Валидационные объекты: признаки (643, 4), цель (643,)
Тестовые объекты: признаки (643, 4), цель (643,)


## Исследуйте модели

### Решающее дерево

Рассмотрим применение модели "Решающее дерево" и подберем для нее оптимальный параметр "максимальная глубина" исходя из критерия максимального значения метрики качества Accuracy для валидационной авборки. Максимальную глубину будем менять в диапазоне 1-5.

In [5]:
best_model_1 = None
best_result_1 = 0
for depth_1 in range(1, 6):
    model_1 = DecisionTreeClassifier(random_state=666, max_depth=depth_1)
    model_1.fit(features_train, target_train)
    predictions_1 = model_1.predict(features_valid)
    result_1 = accuracy_score(target_valid, predictions_1)
    if result_1 > best_result_1:
        best_model_1 = model_1
        best_result_1 = result_1
        best_depth_1 = depth_1
        
print("Accuracy лучшей модели на валидационной выборке:", best_result_1, "Максимальная глубина:", best_depth_1)

Accuracy лучшей модели на валидационной выборке: 0.7853810264385692 Максимальная глубина: 3


### Случайный лес

Рассмотрим применение модели "Случайный лес" и подберем для нее оптимальные параметры "максимальная глубина" и "колличество деревьев" исходя из критерия максимального значения метрики качества Accuracy для валидационной авборки. Максимальную глубину будем ненять в диапазоне 1-5, количество деревьев 1-10.

In [6]:
best_model_2 = None
best_result_2 = 0
best_est = 0
best_depth_2 = 0
for est in range(1, 10):
    for depth_2 in range (1, 6):
        model_2 = RandomForestClassifier(random_state=666, n_estimators=est, max_depth=depth_2) 
        model_2.fit(features_train, target_train) 
        predictions_2 = model_2.predict(features_valid)
        result_2 = accuracy_score(target_valid, predictions_2) 
        if result_2 > best_result_2:
            best_model_2 = model_2
            best_result_2 = result_2
            best_est = est
            best_depth_2 = depth_2

print("Accuracy наилучшей модели на валидационной выборке:", best_result_2, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth_2)


Accuracy наилучшей модели на валидационной выборке: 0.7962674961119751 Количество деревьев: 3 Максимальная глубина: 4


### Логистическая регрессия

Рассмотрим применение модели "Логистическая регрессия" и подберем для нее оптимальный параметр "Алгоритм оптимизации" исходя из критерия максимального значения метрики качества Accuracy для валидационной авборки. Алгоритмы оптимизации: 'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'

In [7]:
best_model_3 = None
best_result_3 = 0
best_sol = ''
for sol in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
    model_3 = LogisticRegression(random_state=666,solver=sol, max_iter=100000)  # инициализируйте модель логистической регрессии с параметром random_state=12345
    model_3.fit(features_train, target_train) # обучите модель на тренировочной выборке
    result_3 = model_3.score(features_valid, target_valid) # получите метрику качества модели на валидационной выборке
    if result_3 > best_result_3:
        best_model_3 = model_3
        best_result_3 = result_3
        best_sol = sol
print("Accuracy наилучшей модели логистической регрессии на валидационной выборке:", best_result_3, "Алгоритм оптимизации:", best_sol)

C:\Users\Lexa\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Lexa\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


Accuracy наилучшей модели логистической регрессии на валидационной выборке: 0.7558320373250389 Алгоритм оптимизации: newton-cg


### Выводы

Среди исследуемых моделей наилучший результат Accuracy (0.796) на валидационной выборке показала модель "Случайный лес" с количеством деревьев - 3 и максимальной глубиной - 4. Худший показатель у модели "Логистическая регрессия" - 0.76


In [8]:
# Попробуем подбор параметров с помощью GridSearchCV
model_2_1 = RandomForestClassifier()
parametrs = { 'n_estimators': range (1, 10),
              'max_depth': range (1,10),
              'min_samples_leaf': range (1,6),
              'min_samples_split': range (2,6,2) }
grid = GridSearchCV(model_2_1, parametrs, cv=3)
grid.fit(features_train, target_train)
grid.best_params_

{'max_depth': 8,
 'min_samples_leaf': 5,
 'min_samples_split': 4,
 'n_estimators': 6}

## Проверьте модель на тестовой выборке

Посчитаем Accuracy для тестовой выборки и сравним с Accuracy для валидационной выборки

### Решающее дерево

In [9]:
test_predictions_1 = best_model_1.predict(features_test)
test_result_1 = accuracy_score(target_test, test_predictions_1)
print("Accuracy для валидационной выборки",best_result_1)
print("Accuracy для тестовой выборки",test_result_1)

Accuracy для валидационной выборки 0.7853810264385692
Accuracy для тестовой выборки 0.7791601866251944


### Случайный лес

In [10]:
test_predictions_2 = best_model_2.predict(features_test)
test_result_2 = accuracy_score(target_test, test_predictions_2)

best_model_2_1 = grid.best_estimator_
test_predictions_2_1 = best_model_2_1.predict(features_test)
test_result_2_1 = accuracy_score(target_test, test_predictions_2_1)
print("Accuracy для валидационной выборки",best_result_2)
print("Accuracy для тестовой выборки",test_result_2)
print("Accuracy для тестовой выборки (модель с GridSearchCV)",test_result_2_1)


Accuracy для валидационной выборки 0.7962674961119751
Accuracy для тестовой выборки 0.7978227060653188
Accuracy для тестовой выборки (модель с GridSearchCV) 0.7869362363919129


### Логистическая регрессия

In [11]:
test_predictions_3 = best_model_3.predict(features_test)
test_result_3 = accuracy_score(target_test, test_predictions_3)
print("Accuracy для валидационной выборки",best_result_3)
print("Accuracy для тестовой выборки",test_result_3)

Accuracy для валидационной выборки 0.7558320373250389
Accuracy для тестовой выборки 0.7387247278382582


### Вывод

Accuracy валидационной и тестовой выборки для моделей "Решающее дерево" и "Случайный лес"  практически совпадают с точностью до 1 %. Модель "Логистическая регрессия" показала на 4 % лучший результат.

In [12]:
result = {'Модель': ['Решающее дерево', 'Случайный лес', 'Логистическая регрессия'], 
          'Accuracy для валидационной выборки': [best_result_1, best_result_2, best_result_3],
          'Accuracy для тестовой выборки': [test_result_1, test_result_2, test_result_3]}
test_result = pd.DataFrame(result)
test_result

,Модель,Accuracy для валидационной выборки,Accuracy для тестовой выборки
0,Решающее дерево,0.785381,0.779160
1,Случайный лес,0.796267,0.797823
2,Логистическая регрессия,0.755832,0.738725


## (бонус) Проверьте модели на адекватность

Критерием адекватности будем считать более высокий показатель Accuracy для моделей в сравнении с показатемем Accuracy для консатанты (пусть для всех объкетов результат предсказания равен 0).

In [13]:

predictions_0 = pd.Series(0, index=target_test.index)
test_result_0 = accuracy_score(target_test, predictions_0)
print("Accuracy для констатны",test_result_0)
print("Accuracy для тестовой выборки модели 'Решающее дерево'",test_result_1)
print("Accuracy для тестовой выборки модели 'Случайный лес'",test_result_2)
print("Accuracy для тестовой выборки модели 'Логистическая регрессия'",test_result_3)

Accuracy для констатны 0.6842923794712286
Accuracy для тестовой выборки модели 'Решающее дерево' 0.7791601866251944
Accuracy для тестовой выборки модели 'Случайный лес' 0.7978227060653188
Accuracy для тестовой выборки модели 'Логистическая регрессия' 0.7387247278382582


### Вывод

Все исследуемые модели дают предсказания с Accuracy выше чем предсказания с константой. Поэтому можно сделать вывод об адекватности моделей. 

## Общий вывод

Построена модель для рекомендации тарифов. Модель обучена на данных о поведении клиентов, которые уже пользуются этими тарифами. Лучшие результат показала модель "Случайный лес" с метрикой качества Accuracy 0.8

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
